In [ ]:
!pip install tensorboard datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

import numpy as np
import pandas as pd

from datasets import load_dataset
from PIL import Image

In [ ]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_dir = "./data"
celeba = load_dataset("tpremoli/CelebA-attrs", split="all", cache_dir=data_dir)

df = celeba.to_pandas()

attr_names = df.columns[1:-1]
image_col_name = df.columns[0]

attributes = df[attr_names].values.astype(np.float32)

inputs = df[image_col_name].values

targets = (attributes + 1) / 2

train_percentage = 0.8
val_percentage = 0.1
test_percentage = 1 - train_percentage - val_percentage

training_amount = int(train_percentage * len(inputs))
validation_amount = int(val_percentage * len(inputs))
testing_amount = int(train_percentage * len(inputs))

train_inputs = inputs[0:training_amount]
val_inputs = inputs[training_amount:training_amount+validation_amount]
test_inputs = inputs[training_amount+validation_amount:]


train_targets = targets[0:training_amount]
val_targets = targets[training_amount:training_amount+validation_amount]
test_targets = targets[training_amount+validation_amount:]

train_targets = torch.tensor(train_targets, dtype=torch.float32).to(device)
val_targets = torch.tensor(val_targets, dtype=torch.float32).to(device)
test_targets = torch.tensor(test_targets, dtype=torch.float32).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/162770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19962 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19867 [00:00<?, ? examples/s]

In [ ]:
from PIL import Image
from io import BytesIO

# Define custom transform
custom_transform = transforms.Compose([
    transforms.CenterCrop((178, 178)),
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

from torch.utils.data import DataLoader, Dataset

class ViTDataset(Dataset):
    def __init__(self, images, targets, feature_extractor):
        self.images = images  # List of image paths or byte data dictionaries
        self.targets = targets  # Corresponding targets
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_data = self.images[idx]

        if isinstance(image_data, dict):  # If image data is stored as a dict (e.g., {'bytes': ...})
            img_bytes = image_data['bytes']
            image = Image.open(BytesIO(img_bytes))
        else:  # Otherwise, assume it's a file path
            image = Image.open(image_data)

        # Apply feature extractor
        encoded_input = self.feature_extractor(images=image, return_tensors="pt")
        pixel_values = encoded_input['pixel_values'].squeeze(0)  # Remove the batch dimension

        target = self.targets[idx]
        return pixel_values, target

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for images, targets in train_loader:
            images = images.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits  # Access logits directly
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        eval_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for images, targets in val_loader:
                images = images.to(device)
                targets = targets.to(device)

                outputs = model(images).logits  # Access logits directly
                loss = criterion(outputs, targets)
                eval_loss += loss.item()

                predictions = torch.sigmoid(outputs)  # Apply sigmoid to get probabilities
                predicted_labels = (predictions > 0.5).float()  # Convert to binary labels

                correct_predictions += (predicted_labels == targets).sum().item()
                total_predictions += targets.numel()

        avg_eval_loss = eval_loss / len(val_loader)
        accuracy = correct_predictions / total_predictions

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Eval Loss: {avg_eval_loss:.4f}, Accuracy: {accuracy:.4f}")

        scheduler.step()

    return model

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification

# Load the pre-trained ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
vit_model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=40)
vit_model.to(device)

# Create datasets and dataloaders
vit_train_dataset = ViTDataset(train_inputs, train_targets, feature_extractor)
vit_val_dataset = ViTDataset(val_inputs, val_targets, feature_extractor)

vit_train_loader = DataLoader(vit_train_dataset, batch_size=16, shuffle=True)
vit_val_loader = DataLoader(vit_val_dataset, batch_size=16, shuffle=False)

# Optimizer and scheduler
optimizer = optim.AdamW(vit_model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = ExponentialLR(optimizer, gamma=0.99)

# Train the ViT model
trained_vit_model = train_model(vit_model, vit_train_loader, vit_val_loader, nn.BCEWithLogitsLoss(), optimizer, scheduler)

# Save the trained ViT model
torch.save(trained_vit_model.state_dict(), "./trained/AnoMed/vit_model.pth")


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/100, Train Loss: 0.2058, Eval Loss: 0.1920, Accuracy: 0.9144
Epoch 2/100, Train Loss: 0.1735, Eval Loss: 0.1876, Accuracy: 0.9168
Epoch 3/100, Train Loss: 0.1548, Eval Loss: 0.1911, Accuracy: 0.9155


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

